In [41]:
import pandas as pd

df = pd.read_csv('ventas.csv')
#print(df.head())

In [42]:
df_pivot = df.pivot_table(
    index=['codigo_unico', 'fecha', 'region', 'sector'], 
    columns='product_name', 
    values='units', 
    fill_value=0
).reset_index()

#print(df_pivot.head())

In [44]:
df_pivot['adopcion'] = df_pivot['suk2'] / (df_pivot['suk2'] +  (0.7 * df_pivot['suk1']))

# Manejar casos donde ambos son 0 para evitar errores de división
df_pivot['adopcion'] = df_pivot['adopcion'].fillna(0)
#print(df_pivot.head())

In [45]:
df_pivot['fecha_str'] = df_pivot['fecha'].astype(str)

# Extraemos Año (primeros 4) y Mes (últimos 2)
df_pivot['año'] = df_pivot['fecha_str'].str[:4].astype(int)
df_pivot['mes'] = df_pivot['fecha_str'].str[-2:].astype(int)

# Creamos una columna de fecha real (día 1 por defecto)
df_pivot['fecha_dt'] = pd.to_datetime(df_pivot['año'].astype(str) + '-' + df_pivot['mes'].astype(str) + '-01')
df_pivot.drop(columns=['fecha_str', 'año', 'mes'], inplace=True)
#print(df_pivot.head())

In [46]:
inicio_promo = pd.to_datetime('2026-01-01')

# Calculamos los meses de promoción de forma numérica
df_pivot['meses_promo'] = (df_pivot['fecha_dt'].dt.year - inicio_promo.year) * 12 + (df_pivot['fecha_dt'].dt.month - inicio_promo.month) + 1
df_pivot.loc[df_pivot['meses_promo'] < 0, 'meses_promo'] = 0
#print(df_pivot.head())

In [47]:
from sklearn.preprocessing import LabelEncoder

# Creamos un diccionario para guardar los encoders por si luego queremos "traducir" de vuelta
encoders = {}

# 1. Procesar Codigo Unico
le_codigo_unico = LabelEncoder()
df_pivot['codigo_unico_encoded'] = le_codigo_unico.fit_transform(df_pivot['codigo_unico'])
encoders['codigo_unico'] = le_codigo_unico

# 2. Procesar Region
le_region = LabelEncoder()
df_pivot['region_encoded'] = le_region.fit_transform(df_pivot['region'])
encoders['region'] = le_region

# 3. Procesar Sector
le_sector = LabelEncoder()
df_pivot['sector_encoded'] = le_sector.fit_transform(df_pivot['sector'])
encoders['sector'] = le_sector

In [48]:
df_pivot['año_temp'] = df_pivot['fecha'].astype(str).str[:4].astype(int)

total_2024 = df_pivot[df_pivot['año_temp'] == 2024].groupby('codigo_unico')[['suk1', 'suk2']].sum().sum(axis=1).reset_index()
total_2024.columns = ['codigo_unico', 'volumen_total_2024']

total_2025 = df_pivot[df_pivot['año_temp'] == 2025].groupby('codigo_unico')[['suk1', 'suk2']].sum().sum(axis=1).reset_index()
total_2025.columns = ['codigo_unico', 'volumen_total_2025']

df_pivot = df_pivot.merge(total_2024, on='codigo_unico', how='left').merge(total_2025, on='codigo_unico', how='left')

df_pivot['volumen_total_2024'] = df_pivot['volumen_total_2024'].fillna(0)
df_pivot['volumen_total_2025'] = df_pivot['volumen_total_2025'].fillna(0)
df_pivot.drop(columns=['año_temp'], inplace=True)

In [49]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

features = [
    'codigo_unico_encoded', 
    'region_encoded', 
    'sector_encoded', 
    'meses_promo',
    'volumen_total_2024',
    'volumen_total_2025' 
]

X = df_pivot[features]
y = df_pivot['adopcion']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definimos los parámetros que queremos probar
param_grid = {
    'n_estimators': [100, 200, 300],      # Número de árboles
    'max_depth': [None, 10, 20, 30],      # Profundidad de los árboles
    'min_samples_split': [2, 5, 10],      # Mínimo de datos para crear una rama
    'max_features': ['sqrt', 'log2']      # Cantidad de variables a considerar por árbol
}

# Inicializamos el modelo base
rf = RandomForestRegressor(random_state=42)

# Configuramos la búsqueda (cv=5 significa que hará validación cruzada de 5 pliegues)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, 
                           cv=5, n_jobs=-1, scoring='neg_mean_absolute_error', verbose=2)

grid_search.fit(X_train, y_train)

# El mejor modelo encontrado
best_model = grid_search.best_estimator_

print(f"Mejores parámetros: {grid_search.best_params_}")

# Evaluación final
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\n--- EVALUACIÓN EN TEST ---")
print(f"Error Medio Absoluto (MAE): {mae:.4f}")
print(f"Precisión (R2 Score): {r2:.4f}") # Qué tan cerca de 1 está el modelo

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Mejores parámetros: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 200}

--- EVALUACIÓN EN TEST ---
Error Medio Absoluto (MAE): 0.1619
Precisión (R2 Score): 0.1560


#### Prediccion

In [50]:
df_proyeccion = df_pivot.drop_duplicates(subset=['codigo_unico']).copy()
df_proyeccion['meses_promo'] = 5

X_futuro = df_proyeccion[features] # Las mismas que usaste en el entrenamiento
df_proyeccion['adopcion_predicho_M5'] = best_model.predict(X_futuro)

# Traducimos los códigos a nombres reales para que el reporte sea útil
df_proyeccion['codigo_unico_original'] = encoders['codigo_unico'].inverse_transform(df_proyeccion['codigo_unico_encoded'])

In [51]:
# ¿Cuántas cuentas alcanzarán el objetivo del 70% en el mes 5?
objetivo_cumplido = df_proyeccion[df_proyeccion['adopcion_predicho_M5'] >= 0.70]
porcentaje_exito = (len(objetivo_cumplido) / len(df_proyeccion)) * 100

print(f"En el mes 5, se predice que el {porcentaje_exito:.1f}% de las cuentas alcanzarán el objetivo.")

# ¿Cuál será el adopcion promedio por region?
impacto_zonas = df_proyeccion.groupby('region')['adopcion_predicho_M5'].mean()
#print("\nImpacto esperado por region en el Mes 5:")
#print(impacto_zonas)

En el mes 5, se predice que el 12.7% de las cuentas alcanzarán el objetivo.


In [52]:
# 1. Seleccionamos las columnas más importantes para que el Excel sea limpio
columnas_excel = [
    'codigo_unico_original', 
    'region', 
    'sector', 
    'volumen_total_2024', 
    'volumen_total_2025', 
    'meses_promo', 
    'adopcion_predicho_M5'
]

# 2. Creamos un archivo Excel
df_proyeccion[columnas_excel].to_excel('Proyeccion_Adopcion_Mayo.xlsx', index=False)

print("¡Archivo exportado con éxito!")

¡Archivo exportado con éxito!
